In [388]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Collect data for each feature from CSV files downloaded from web

## HPI : Target Feature

S&P/Case-Shiller U.S. National Home Price Index (Index Jan 2000=100, Seasonally Adjusted). The "S&P/Case-Shiller U.S. National Home Price Index" is a widely recognized and respected measure of changes in residential real estate prices in the United States. It is published by S&P Dow Jones Indices and CoreLogic and is often referred to simply as the "Case-Shiller Index." The index is designed to track the relative changes in the prices of single-family homes over time. Here "Index Jan 2000=100" specifies the base period for the index. It sets the index value for January 2000 as 100, and all subsequent index values are expressed relative to this base. For example, if the index value for a later date is 150, it means that home prices have increased by 50% since January 2000.

In [389]:
hpi = pd.read_csv("data/HPI.csv")

In [390]:
hpi.rename(columns = {"CSUSHPINSA":"HPI"}, inplace = True)

In [391]:
hpi['Date'] = pd.to_datetime(hpi['DATE'])
hpi.drop(["DATE"], axis = 1, inplace = True)

In [392]:
hpi.head()

,HPI,Date
0,100.000,2000-01-01
1,100.571,2000-02-01
2,101.466,2000-03-01
3,102.540,2000-04-01
4,103.701,2000-05-01


In [393]:
hpi["Date"].max()

Timestamp('2023-06-01 00:00:00')

In [394]:
hpi["Date"].min()

Timestamp('2000-01-01 00:00:00')

**hpi has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-06-01**"

## Demand Features

### Interest Rates, Discount Rate for United States

In [395]:
interest = pd.read_csv("data/demand/Interest_discount.csv")

In [396]:
interest.rename(columns = {"INTDSRUSM193N":"InterestDiscount"}, inplace = True)

In [397]:
interest['Date'] = pd.to_datetime(interest['DATE'])
interest.drop(["DATE"], axis = 1, inplace = True)

In [398]:
interest.head()

,InterestDiscount,Date
0,5.00,2000-01-01
1,5.24,2000-02-01
2,5.34,2000-03-01
3,5.50,2000-04-01
4,5.71,2000-05-01


In [399]:
interest["Date"].max()

Timestamp('2021-08-01 00:00:00')

In [400]:
interest["Date"].min()

Timestamp('2000-01-01 00:00:00')

**interest has monthly frequency with minimum date as "2000-01-01" and maximum date as "2021-08-01**"

### 30-Year Fixed Rate Mortgage Average in the United States

In [401]:
mortgage = pd.read_csv("data/demand/Mortgage Avg 30 YRS.csv")

In [402]:
mortgage

,DATE,MORTGAGE30US
0,2000-01-01,8.2100
1,2000-02-01,8.3250
2,2000-03-01,8.2400
3,2000-04-01,8.1525
4,2000-05-01,8.5150
...,...,...
279,2023-04-01,6.3425
280,2023-05-01,6.4250
281,2023-06-01,6.7140
282,2023-07-01,6.8400


In [403]:
mortgage.rename(columns = {"MORTGAGE30US":"Avg Mortgage"}, inplace = True)

In [404]:
mortgage['Date'] = pd.to_datetime(mortgage['DATE'])
mortgage.drop(["DATE"], axis = 1, inplace = True)

In [405]:
mortgage.head()

,Avg Mortgage,Date
0,8.2100,2000-01-01
1,8.3250,2000-02-01
2,8.2400,2000-03-01
3,8.1525,2000-04-01
4,8.5150,2000-05-01


In [406]:
mortgage["Date"].max()

Timestamp('2023-08-01 00:00:00')

In [407]:
mortgage["Date"].min()

Timestamp('2000-01-01 00:00:00')

**Avg Mortgage has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-08-01. We will consider until 2023-06-01**"

### GDP Per Capita

In [408]:
gdp = pd.read_csv("data/demand/GDP.csv")

In [409]:
gdp

,DATE,A939RX0Q048SBEA
0,2000-01-01,45983.0
1,2000-04-01,46704.0
2,2000-07-01,46624.0
3,2000-10-01,46777.0
4,2001-01-01,46519.0
...,...,...
89,2022-04-01,59688.0
90,2022-07-01,60080.0
91,2022-10-01,60376.0
92,2023-01-01,60611.0


In [410]:
gdp.rename(columns = {"A939RX0Q048SBEA":"gdp"}, inplace = True)

In [411]:
gdp['Date'] = pd.to_datetime(gdp['DATE'])
gdp.drop(["DATE"], axis = 1, inplace = True)

In [412]:
gdp.head()

,gdp,Date
0,45983.0,2000-01-01
1,46704.0,2000-04-01
2,46624.0,2000-07-01
3,46777.0,2000-10-01
4,46519.0,2001-01-01


In [413]:
gdp["Date"].max()

Timestamp('2023-04-01 00:00:00')

In [414]:
gdp["Date"].min()

Timestamp('2000-01-01 00:00:00')

**gdp has quarterly frequency with minimum date as "2000-01-01" and maximum date as "2023-04-01". We will consider until 2023-06-01**"

**We need to interpolate gdp to make it monthly frequency**

### Consumer Sentiment

In [415]:
consumer_sentiment = pd.read_csv("data/demand/consmer sentiment.csv")

In [416]:
consumer_sentiment

,Month,Year,Index
0,1,2000,112.0
1,2,2000,111.3
2,3,2000,107.1
3,4,2000,109.2
4,5,2000,110.7
...,...,...,...
278,3,2023,62.0
279,4,2023,63.5
280,5,2023,59.2
281,6,2023,64.4


In [417]:
# Create a datetime column from 'Year' and 'Month'
consumer_sentiment['Date'] = pd.to_datetime(consumer_sentiment[['Year', 'Month']].assign(day=1))

In [418]:
consumer_sentiment.drop(["Month", "Year"], inplace = True, axis = 1)

In [419]:
consumer_sentiment.rename(columns = {"Index":"Cons. Sentiment Index"}, inplace = True)

In [420]:
consumer_sentiment.head()

,Cons. Sentiment Index,Date
0,112.0,2000-01-01
1,111.3,2000-02-01
2,107.1,2000-03-01
3,109.2,2000-04-01
4,110.7,2000-05-01


In [421]:
consumer_sentiment["Date"].max()

Timestamp('2023-07-01 00:00:00')

In [422]:
consumer_sentiment["Date"].min()

Timestamp('2000-01-01 00:00:00')

**consumer sentiment index has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-07-01". We will consider until 2023-06-01**"

### Inflation

In [423]:
cpi = pd.read_csv("data/demand/cpi.csv")
cpi

,DATE,CPALTT01USM657N
0,2000-01-01,0.297089
1,2000-02-01,0.592417
2,2000-03-01,0.824499
3,2000-04-01,0.058411
4,2000-05-01,0.116754
...,...,...
277,2023-02-01,0.558211
278,2023-03-01,0.331073
279,2023-04-01,0.505904
280,2023-05-01,0.251844


In [424]:
cpi.rename(columns = {"CPALTT01USM657N":"cpi"}, inplace = True)


In [425]:
cpi['Date'] = pd.to_datetime(cpi['DATE'])
cpi.drop(["DATE"], axis = 1, inplace = True)

In [426]:
cpi.head()

,cpi,Date
0,0.297089,2000-01-01
1,0.592417,2000-02-01
2,0.824499,2000-03-01
3,0.058411,2000-04-01
4,0.116754,2000-05-01


In [427]:
cpi["Date"].max()

Timestamp('2023-06-01 00:00:00')

In [428]:
cpi["Date"].min()

Timestamp('2000-01-01 00:00:00')

**cpi has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-06-01". We will consider until 2023-06-01**"

### Population

In [429]:
population = pd.read_csv("data/demand/population.csv")
population

,DATE,CNP16OV
0,2000-01-01,211410
1,2000-02-01,211576
2,2000-03-01,211772
3,2000-04-01,212018
4,2000-05-01,212242
...,...,...
279,2023-04-01,266443
280,2023-05-01,266618
281,2023-06-01,266801
282,2023-07-01,267002


In [430]:
population.rename(columns = {"CNP16OV":"population"}, inplace = True)

In [431]:
population['Date'] = pd.to_datetime(population['DATE'])
population.drop(["DATE"], axis = 1, inplace = True)

In [432]:
population.head()

,population,Date
0,211410,2000-01-01
1,211576,2000-02-01
2,211772,2000-03-01
3,212018,2000-04-01
4,212242,2000-05-01


In [433]:
population["Date"].max()

Timestamp('2023-08-01 00:00:00')

In [434]:
population["Date"].min()

Timestamp('2000-01-01 00:00:00')

**population has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-08-01". We will consider until 2023-06-01"**

### employed population

In [435]:
emp_population = pd.read_csv("data/demand/employed population.csv")
emp_population

,DATE,CE16OV
0,2000-01-01,136559
1,2000-02-01,136598
2,2000-03-01,136701
3,2000-04-01,137270
4,2000-05-01,136630
...,...,...
279,2023-04-01,161031
280,2023-05-01,160721
281,2023-06-01,160994
282,2023-07-01,161262


In [436]:
emp_population.rename(columns = {"CE16OV":"employed population"}, inplace = True)

In [437]:
emp_population['Date'] = pd.to_datetime(emp_population['DATE'])
emp_population.drop(["DATE"], axis = 1, inplace = True)

In [438]:
emp_population.head()

,employed population,Date
0,136559,2000-01-01
1,136598,2000-02-01
2,136701,2000-03-01
3,137270,2000-04-01
4,136630,2000-05-01


In [439]:
emp_population["Date"].max()

Timestamp('2023-08-01 00:00:00')

In [440]:
emp_population["Date"].min()

Timestamp('2000-01-01 00:00:00')

**employed population has monthly frequency with minimum date as "2000-01-01" and maximum date as "2023-08-01". We will consider until 2023-06-01"**

### Merging all Demand Data

In [441]:
merged_df = pd.merge(hpi, interest, on='Date', how='left')
merged_df = pd.merge(merged_df, mortgage, on='Date', how='left')
merged_df = pd.merge(merged_df, gdp, on='Date', how='left')
merged_df = pd.merge(merged_df, consumer_sentiment, on='Date', how='left')
merged_df = pd.merge(merged_df, cpi, on='Date', how='left')
merged_df = pd.merge(merged_df, population, on='Date', how='left')
merged_df = pd.merge(merged_df, emp_population, on='Date', how='left')

In [442]:
merged_df

,HPI,Date,InterestDiscount,Avg Mortgage,gdp,Cons. Sentiment Index,cpi,population,employed population
0,100.000,2000-01-01,5.00,8.2100,45983.0,112.0,0.297089,211410,136559
1,100.571,2000-02-01,5.24,8.3250,NaN,111.3,0.592417,211576,136598
2,101.466,2000-03-01,5.34,8.2400,NaN,107.1,0.824499,211772,136701
3,102.540,2000-04-01,5.50,8.1525,46704.0,109.2,0.058411,212018,137270
4,103.701,2000-05-01,5.71,8.5150,NaN,110.7,0.116754,212242,136630
...,...,...,...,...,...,...,...,...,...
277,293.341,2023-02-01,NaN,6.2575,NaN,67.0,0.558211,266112,160315
278,297.127,2023-03-01,NaN,6.5440,NaN,62.0,0.331073,266272,160892
279,301.457,2023-04-01,NaN,6.3425,60852.0,63.5,0.505904,266443,161031
280,305.402,2023-05-01,NaN,6.4250,NaN,59.2,0.251844,266618,160721


## Supply Features

### Avg. House sales parice

In [443]:
avg_sales_price = pd.read_csv("data/supply/Avg. House Sales Price.csv")
avg_sales_price

,DATE,ASPUS
0,2000-01-01,202900.0
1,2000-04-01,202400.0
2,2000-07-01,204100.0
3,2000-10-01,212100.0
4,2001-01-01,211000.0
...,...,...
89,2022-04-01,527500.0
90,2022-07-01,547800.0
91,2022-10-01,552600.0
92,2023-01-01,505300.0


In [444]:
avg_sales_price.rename(columns = {"ASPUS":"avg house sales price"}, inplace = True)

In [445]:
avg_sales_price['Date'] = pd.to_datetime(avg_sales_price['DATE'])
avg_sales_price.drop(["DATE"], axis = 1, inplace = True)

In [446]:
avg_sales_price.head()

,avg house sales price,Date
0,202900.0,2000-01-01
1,202400.0,2000-04-01
2,204100.0,2000-07-01
3,212100.0,2000-10-01
4,211000.0,2001-01-01


In [447]:
avg_sales_price["Date"].max()

Timestamp('2023-04-01 00:00:00')

In [448]:
avg_sales_price["Date"].min()

Timestamp('2000-01-01 00:00:00')

### MOnthly Supply of new houses

In [449]:
monthly_new_houses = pd.read_csv("data/supply/Monthly Supply of New Houses .csv")
monthly_new_houses

,DATE,MSACSR
0,2000-01-01,4.3
1,2000-02-01,4.3
2,2000-03-01,4.3
3,2000-04-01,4.4
4,2000-05-01,4.4
...,...,...
278,2023-03-01,8.1
279,2023-04-01,7.6
280,2023-05-01,7.3
281,2023-06-01,7.5


In [450]:
monthly_new_houses.rename(columns = {"MSACSR":"monthly new houses"}, inplace = True)

In [451]:
monthly_new_houses['Date'] = pd.to_datetime(monthly_new_houses['DATE'])
monthly_new_houses.drop(["DATE"], axis = 1, inplace = True)

In [452]:
monthly_new_houses.head()

,monthly new houses,Date
0,4.3,2000-01-01
1,4.3,2000-02-01
2,4.3,2000-03-01
3,4.4,2000-04-01
4,4.4,2000-05-01


In [453]:
monthly_new_houses["Date"].max()

Timestamp('2023-07-01 00:00:00')

In [454]:
monthly_new_houses["Date"].min()

Timestamp('2000-01-01 00:00:00')

### New  Housing Units Authorized

In [455]:
new_houses_authorized = pd.read_csv("data/supply/New Housing Units Authorized.csv")
new_houses_authorized

,DATE,PERMIT
0,2000-01-01,1727.0
1,2000-02-01,1692.0
2,2000-03-01,1651.0
3,2000-04-01,1597.0
4,2000-05-01,1543.0
...,...,...
278,2023-03-01,1437.0
279,2023-04-01,1417.0
280,2023-05-01,1496.0
281,2023-06-01,1441.0


In [456]:
new_houses_authorized.rename(columns = {"PERMIT":"new houses authorized"}, inplace = True)

In [457]:
new_houses_authorized['Date'] = pd.to_datetime(new_houses_authorized['DATE'])
new_houses_authorized.drop(["DATE"], axis = 1, inplace = True)

In [458]:
new_houses_authorized.head()

,new houses authorized,Date
0,1727.0,2000-01-01
1,1692.0,2000-02-01
2,1651.0,2000-03-01
3,1597.0,2000-04-01
4,1543.0,2000-05-01


In [459]:
new_houses_authorized["Date"].max()

Timestamp('2023-07-01 00:00:00')

In [460]:
new_houses_authorized["Date"].min()

Timestamp('2000-01-01 00:00:00')

### New  Housing Units Completed

In [461]:
new_houses_completed = pd.read_csv("data/supply/New Housing Units Completed.csv")
new_houses_completed

,DATE,COMPUTSA
0,2000-01-01,1574.0
1,2000-02-01,1677.0
2,2000-03-01,1704.0
3,2000-04-01,1610.0
4,2000-05-01,1682.0
...,...,...
278,2023-03-01,1528.0
279,2023-04-01,1416.0
280,2023-05-01,1534.0
281,2023-06-01,1498.0


In [462]:
new_houses_completed.rename(columns = {"COMPUTSA":"new houses completed"}, inplace = True)

In [463]:
new_houses_completed['Date'] = pd.to_datetime(new_houses_completed['DATE'])
new_houses_completed.drop(["DATE"], axis = 1, inplace = True)

In [464]:
new_houses_completed.head()

,new houses completed,Date
0,1574.0,2000-01-01
1,1677.0,2000-02-01
2,1704.0,2000-03-01
3,1610.0,2000-04-01
4,1682.0,2000-05-01


In [465]:
new_houses_completed["Date"].max()

Timestamp('2023-07-01 00:00:00')

In [466]:
new_houses_completed["Date"].min()

Timestamp('2000-01-01 00:00:00')

### producer price index

In [467]:
ppi = pd.read_csv("data/supply/producer price index.csv")
ppi

,DATE,WPUSI012011
0,2000-01-01,144.100
1,2000-02-01,144.700
2,2000-03-01,145.400
3,2000-04-01,145.600
4,2000-05-01,144.900
...,...,...
278,2023-03-01,331.729
279,2023-04-01,333.549
280,2023-05-01,338.154
281,2023-06-01,337.953


In [468]:
ppi.rename(columns = {"WPUSI012011":"ppi"}, inplace = True)

In [469]:
ppi['Date'] = pd.to_datetime(ppi['DATE'])
ppi.drop(["DATE"], axis = 1, inplace = True)

In [470]:
ppi.head()

,ppi,Date
0,144.1,2000-01-01
1,144.7,2000-02-01
2,145.4,2000-03-01
3,145.6,2000-04-01
4,144.9,2000-05-01


In [471]:
ppi["Date"].max()

Timestamp('2023-07-01 00:00:00')

In [472]:
ppi["Date"].min()

Timestamp('2000-01-01 00:00:00')

### Merging all Supply Data

In [473]:
merged_df = pd.merge(merged_df, avg_sales_price, on='Date', how='left')
merged_df = pd.merge(merged_df, monthly_new_houses, on='Date', how='left')
merged_df = pd.merge(merged_df, new_houses_authorized, on='Date', how='left')
merged_df = pd.merge(merged_df, new_houses_completed, on='Date', how='left')
merged_df = pd.merge(merged_df, ppi, on='Date', how='left')

In [474]:
merged_df

,HPI,Date,InterestDiscount,Avg Mortgage,gdp,Cons. Sentiment Index,cpi,population,employed population,avg house sales price,monthly new houses,new houses authorized,new houses completed,ppi
0,100.000,2000-01-01,5.00,8.2100,45983.0,112.0,0.297089,211410,136559,202900.0,4.3,1727.0,1574.0,144.100
1,100.571,2000-02-01,5.24,8.3250,NaN,111.3,0.592417,211576,136598,NaN,4.3,1692.0,1677.0,144.700
2,101.466,2000-03-01,5.34,8.2400,NaN,107.1,0.824499,211772,136701,NaN,4.3,1651.0,1704.0,145.400
3,102.540,2000-04-01,5.50,8.1525,46704.0,109.2,0.058411,212018,137270,202400.0,4.4,1597.0,1610.0,145.600
4,103.701,2000-05-01,5.71,8.5150,NaN,110.7,0.116754,212242,136630,NaN,4.4,1543.0,1682.0,144.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,293.341,2023-02-01,NaN,6.2575,NaN,67.0,0.558211,266112,160315,NaN,8.4,1482.0,1577.0,330.532
278,297.127,2023-03-01,NaN,6.5440,NaN,62.0,0.331073,266272,160892,NaN,8.1,1437.0,1528.0,331.729
279,301.457,2023-04-01,NaN,6.3425,60852.0,63.5,0.505904,266443,161031,495100.0,7.6,1417.0,1416.0,333.549
280,305.402,2023-05-01,NaN,6.4250,NaN,59.2,0.251844,266618,160721,NaN,7.3,1496.0,1534.0,338.154


In [475]:
merged_df.to_csv("Complete_data_unclean.csv")

In [476]:
merged_data = pd.read_csv("complete_data_unclean.csv")

In [477]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 0 to 281
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   HPI                    282 non-null    float64       
 1   Date                   282 non-null    datetime64[ns]
 2   InterestDiscount       260 non-null    float64       
 3   Avg Mortgage           282 non-null    float64       
 4   gdp                    94 non-null     float64       
 5   Cons. Sentiment Index  282 non-null    float64       
 6   cpi                    282 non-null    float64       
 7   population             282 non-null    int64         
 8   employed population    282 non-null    int64         
 9   avg house sales price  94 non-null     float64       
 10  monthly new houses     282 non-null    float64       
 11  new houses authorized  282 non-null    float64       
 12  new houses completed   282 non-null    float64       
 13  ppi  

### Things to be done to deal with missing data:
- need to fill na in InterestDiscount
- Interpolate gdp(quaterly to monthly)
- Interpolate avg house sales price(quaterly to monthly)

**Since we have time series data we will use time based interpolation**

In [478]:
merged_df = merged_df.set_index(merged_df["Date"], drop = False)

In [479]:
merged_df["gdp"] = merged_df["gdp"].interpolate(method='time')

In [480]:
merged_df["InterestDiscount"] = merged_df["InterestDiscount"].interpolate(method='time')

In [481]:
merged_df["avg house sales price"] = merged_df["avg house sales price"].interpolate(method='time')

In [482]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 282 entries, 2000-01-01 to 2023-06-01
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   HPI                    282 non-null    float64       
 1   Date                   282 non-null    datetime64[ns]
 2   InterestDiscount       282 non-null    float64       
 3   Avg Mortgage           282 non-null    float64       
 4   gdp                    282 non-null    float64       
 5   Cons. Sentiment Index  282 non-null    float64       
 6   cpi                    282 non-null    float64       
 7   population             282 non-null    int64         
 8   employed population    282 non-null    int64         
 9   avg house sales price  282 non-null    float64       
 10  monthly new houses     282 non-null    float64       
 11  new houses authorized  282 non-null    float64       
 12  new houses completed   282 non-null    float6

In [483]:
merged_df.to_csv("Complete_data.csv")

In [484]:
merged_df.isna().sum()

HPI                      0
Date                     0
InterestDiscount         0
Avg Mortgage             0
gdp                      0
Cons. Sentiment Index    0
cpi                      0
population               0
employed population      0
avg house sales price    0
monthly new houses       0
new houses authorized    0
new houses completed     0
ppi                      0
dtype: int64

### We have complete dataset now